In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from torchtext import data
from torchtext import datasets

import numpy as np
from sklearn.model_selection import KFold 

import time
import random

In [2]:
SEED = 1234

# We have looked at https://github.com/bentrevett/pytorch-sentiment-analysis for setting the various values of TEXT and LABEL as we have not worked with this before till now.

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
train_data, _ = datasets.IMDB.splits(TEXT, LABEL)                                   #split the data into training and test data
train_data, val_data= train_data.split(split_ratio=0.8, random_state=random.seed(SEED))     #split training data into training and validation data

print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(val_data)}')


aclImdb_v1.tar.gz:   0%|          | 98.3k/84.1M [00:00<01:27, 963kB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 51.0MB/s]


In [ ]:
# Testing data from SST dataset
_ , _ , test_data = datasets.SST.splits(TEXT, LABEL)

print(f'Number of testing examples: {len(test_data)}')

downloading trainDevTestTrees_PTB.zip


trainDevTestTrees_PTB.zip: 100%|██████████| 790k/790k [00:00<00:00, 1.08MB/s]


extracting
Number of testing examples: 2210


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(vars(train_data.examples[0]))

{'text': ['So', 'you', 'might', 'be', 'reading', 'some', 'of', 'the', 'comments', 'posted', 'on', 'this', 'film', ',', 'and', 'you', 'might', 'be', 'thinking', 'to', 'yourself', ',', '"', 'Huh', '.', 'There', 'were', 'sure', 'a', 'bunch', 'of', 'RAVE', 'REVIEWS', 'posted', 'on', 'December', '30', '.', '"', 'Funny', 'thing', 'is', ',', 'most', 'of', 'these', 'rave', 'reviews', 'sound', 'like', 'they', "'re", 'coming', 'from', 'the', 'same', 'person', ',', 'or', 'at', 'best', 'a', 'small', ',', 'coordinated', 'group', 'of', '"', 'Open', 'House', '"', 'groupies', '.', 'The', 'truth', ',', 'my', 'friends', ',', 'is', 'that', 'this', 'film', 'is', 'truly', 'unwatchable', '.', 'Just', 'because', 'it', "'s", '"', 'independent', '"', 'does', "n't", 'mean', 'it', 'gets', 'a', 'free', 'pass', '.', 'If', 'you', "'re", 'going', 'to', 'make', 'a', 'musical', ',', 'whether', 'on', 'film', 'or', 'on', 'stage', ',', 'whether', 'on', 'Broadway', 'or', 'at', 'the', 'local', 'community', 'playhouse', ','

In [ ]:
MAX_VOCAB_SIZE = 50000 #consider only the top 50000 used words to build vocabulary/embeddings

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

.vector_cache/glove.6B.zip: 862MB [06:29, 2.22MB/s]                          
100%|█████████▉| 399273/400000 [00:17<00:00, 22804.14it/s]

Unique tokens in TEXT vocabulary: 50002
Unique tokens in LABEL vocabulary: 2


In [ ]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 231460), (',', 220381), ('.', 189521), ('and', 125033), ('a', 124956), ('of', 115251), ('to', 107051), ('is', 87411), ('in', 69978), ('I', 62093), ('it', 61264), ('that', 56349), ('"', 50913), ("'s", 49710), ('this', 48489), ('-', 42042), ('/><br', 40743), ('was', 40065), ('as', 34656), ('with', 34180)]


In [ ]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [ ]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f1302a3a510>, {'neg': 0, 'pos': 1})


In [ ]:
BATCH_SIZE = 64

train_iterator, val_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, val_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [ ]:
#This model has been built from scratch without using the links provided in the doc. The basic framework is taken from https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

class BiLSTM(nn.Module):
    def __init__(self,input_dim,embedding_dim,hidden_dim,output_dim,n_layers,bidirectional,dropout,pad_idx):
        super().__init__()

        self.embedding=nn.Embedding(input_dim,embedding_dim,padding_idx=pad_idx)

        self.lstm=nn.LSTM(embedding_dim, hidden_dim,num_layers=n_layers,bidirectional=bidirectional,dropout=dropout if n_layers>1 else 0)

        #self.gru=nn.GRU(embedding_dim, hidden_dim,num_layers=n_layers,bidirectional=bidirectional,dropout=dropout if n_layers>1 else 0)

        self.fc=nn.Linear(hidden_dim, output_dim)

        self.dropout=nn.Dropout(dropout)

    def forward(self,text,text_lengths):
        embed=self.embedding(text)
        #text_lengths = torch.as_tensor(text_lengths, dtype=torch.int64, device='cpu')
        #packed_embedded = nn.utils.rnn.pack_padded_sequence(embed, text_lengths) uncomment to introduce padding of sequence
        #packed_output, (hidden, cell) = self.lstm(packed_embedded)               uncomment to train lstm with padding
        #packed_output, (hidden, cell) = self.gru(packed_embedded)                uncomment to train gru with padding
        output, (hidden,cell) = self.lstm(embed)  
        #output, hidden = self.gru(embed)                                         uncomment for normal gru
        #output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output) uncomment if using padding
        #print(hidden.shape)
        hidden=self.dropout(hidden)
        #hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)) uncomment if using bidirectional lstm only (not gru)
        return self.fc(hidden)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM=1
#OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 1
BIDIRECTIONAL = False #keep true if using bidirectional GRU/LSTM
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
print(INPUT_DIM)
print(OUTPUT_DIM)

50002
1


In [ ]:
model=BiLSTM(INPUT_DIM,EMBEDDING_DIM,HIDDEN_DIM,OUTPUT_DIM,N_LAYERS,BIDIRECTIONAL,DROPOUT,PAD_IDX)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
#loss_function = nn.NLLLoss()
loss_function=nn.BCEWithLogitsLoss()
loss_function.to(device)

BCEWithLogitsLoss()

In [ ]:
#model.embedding.weight.data.copy_(pretrained_embeddings)

In [ ]:
# We have used this function directly from https://github.com/bentrevett/pytorch-sentiment-analysis but there is nothing new about this as this is a standard implementation

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
num_epochs=30

trainloss=[]
trainacc=[]
valacc=[]
valloss=[]
testacc=[]
testloss=[]


#We have used the training and testing loops based on the pytorch link mentioned before. Few changes are there for dimension matching etc.

for epoch in range(num_epochs):
  start_time=time.time()
  model.train()


################################################ TRAINING ################################################

  train_acc=0
  train_loss=0
  for batch in train_iterator:
    model.zero_grad()
    #print(batch.text.shape)
    text, text_lengths= batch.text
    #print(sentiment.shape)
    sentiment=model(text, text_lengths).squeeze(1)
    sentiment=torch.reshape(sentiment,(len(batch.label),))
    #print(sentiment.shape)
    #print(batch.label.shape)
    loss=loss_function(sentiment,batch.label)
    loss.backward()
    optimizer.step()
    train_loss=train_loss+loss.item()
    #_,indices=torch.max(sentiment,1)
    #train_acc=train_acc+torch.mean(torch.tensor(batch.label == indices, dtype=torch.float))
    train_acc=train_acc+binary_accuracy(sentiment, batch.label)

  train_loss=train_loss/len(train_iterator)
  train_acc=train_acc/len(train_iterator)
  trainloss.append(float(train_loss))
  trainacc.append(float(train_acc))

  val_acc=0
  val_loss=0
  model.eval()


############################################################# VALIDATION/TEST ###############################################


  with torch.no_grad():
    for batch in val_iterator:
      text, text_lengths= batch.text
      text_lengths = torch.as_tensor(text_lengths, dtype=torch.int64, device='cpu')
      prediction=model(text, text_lengths).squeeze(1)
      prediction=torch.reshape(prediction,(len(batch.label),))
      val_loss=val_loss+loss_function(prediction,batch.label).item()
      #_,indices=torch.max(prediction,1)
      #val_acc=val_acc+torch.mean(torch.tensor(batch.label == indices, dtype=torch.float))
      val_acc=val_acc+binary_accuracy(prediction, batch.label)
  val_loss=val_loss/len(val_iterator)
  val_acc=val_acc/len(val_iterator)
  valloss.append(float(val_loss))
  valacc.append(float(val_acc))


  test_acc=0
  test_loss=0
  model.eval()

  with torch.no_grad():
    for batch in test_iterator:
      text, text_lengths= batch.text
      text_lengths = torch.as_tensor(text_lengths, dtype=torch.int64, device='cpu')
      prediction=model(text, text_lengths).squeeze(1)
      prediction=torch.reshape(prediction,(len(batch.label),))
      test_loss=test_loss+loss_function(prediction,batch.label).item()
      #_,indices=torch.max(prediction,1)
      #val_acc=val_acc+torch.mean(torch.tensor(batch.label == indices, dtype=torch.float))
      test_acc=test_acc+binary_accuracy(prediction, batch.label)

  test_loss=test_loss/len(test_iterator)
  test_acc=test_acc/len(test_iterator)
  testloss.append(float(test_loss))
  testacc.append(float(test_acc))

  end_time=time.time()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')
  print(f'\tTest Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


In [ ]:
# This cell is used to see the final test accuracy
test_acc=0
test_loss=0
model.eval()

with torch.no_grad():
  for batch in test_iterator:
    text, text_lengths= batch.text
    text_lengths = torch.as_tensor(text_lengths, dtype=torch.int64, device='cpu')
    prediction=model(text, text_lengths).squeeze(1)
    prediction=torch.reshape(prediction,(len(batch.label),))
    test_loss=test_loss+loss_function(prediction,batch.label).item()
    #_,indices=torch.max(prediction,1)
    #val_acc=val_acc+torch.mean(torch.tensor(batch.label == indices, dtype=torch.float))
    test_acc=test_acc+binary_accuracy(prediction, batch.label)

test_loss=test_loss/len(test_iterator)
test_acc=test_acc/len(test_iterator)
testloss.append(float(test_loss))
testacc.append(float(test_acc))

end_time=time.time()

epoch_mins, epoch_secs = epoch_time(start_time, end_time)

print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')
print(f'\tTest Loss: {test_loss:.3f} | Testn Acc: {test_acc*100:.2f}%')


Epoch: 05 | Epoch Time: 0m 8s
	Train Loss: 58.013 | Train Acc: 6623.44%
	 Val. Loss: 0.624 |  Val. Acc: 66.99%
	Test Loss: 0.515 | Testn Acc: 70.86%


In [ ]:
import csv

rows=zip(trainloss,trainacc,valloss,valacc,testacc,testloss)
with open("lstm.csv", "w") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)

In [ ]:
# References
# https://github.com/bentrevett/pytorch-sentiment-analysis
# https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html